# Day19_1: RAG (Retrieval-Augmented Generation) - 검색 증강 생성 (정답)

## 학습 목표

**Part 1: 기초**
1. LLM의 한계(Hallucination, 지식 단절) 이해하기
2. RAG 개념과 파이프라인 이해하기
3. 텍스트 임베딩 이해하기
4. 벡터 데이터베이스 개념 이해하기
5. 유사도 검색(Similarity Search) 이해하기

**Part 2: 심화**
1. 문서 청킹(Chunking) 전략 이해하기
2. LangChain으로 RAG 구현하기
3. 프롬프트 템플릿 활용하기
4. AI 뉴스 Q&A 봇 실습하기

---

## 실습 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import os
import numpy as np
from dotenv import load_dotenv

# LangChain 관련
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

# 환경 변수 로드
load_dotenv()

# 임베딩 및 LLM 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

print("환경 설정 완료!")

In [ ]:
# 본문 예제에서 사용한 벡터 DB 생성 (퀴즈 풀이용)
# 샘플 문서 생성
documents = [
    Document(page_content="2024년 매출은 100억원입니다.", metadata={"source": "재무보고서"}),
    Document(page_content="전년 대비 매출이 20% 증가했습니다.", metadata={"source": "재무보고서"}),
    Document(page_content="직원 복지로 매년 휴가비를 지원합니다.", metadata={"source": "사내규정"}),
    Document(page_content="연차는 입사 1년 후부터 15일이 부여됩니다.", metadata={"source": "사내규정"}),
    Document(page_content="신제품 AI 스피커가 3월에 출시됩니다.", metadata={"source": "뉴스"}),
]

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="demo_collection"
)

# 뉴스 기사 벡터 DB
news_articles = [
    {
        "title": "AI, 신약 개발 속도 10배 앞당긴다",
        "content": "2025년, 인공지능(AI)이 신약 개발 분야에서 혁신을 주도하고 있다. 전통적으로 10년 이상 소요되던 신약 개발 기간이 AI 기반 시뮬레이션과 데이터 분석을 통해 평균 1~2년으로 단축될 전망이다. 글로벌 제약사 '뉴로젠'은 최근 AI 플랫폼 '제네시스-1'을 활용해 알츠하이머 치료제 후보 물질을 6개월 만에 발굴했다고 발표했다.",
        "source": "AI 타임즈",
        "date": "2025-01-15"
    },
    {
        "title": "데이터브릭스, 클라우드 비용 최적화 AI 스타트업 '옵티마이즈' 인수",
        "content": "데이터 및 AI 기업 데이터브릭스가 클라우드 비용 관리 AI 스타트업 '옵티마이즈(Optimize.AI)'를 5억 달러에 인수한다고 발표했다. 옵티마이즈는 AI를 사용해 기업의 클라우드 사용 패턴을 실시간으로 분석하고, 불필요한 자원을 자동으로 축소하여 비용을 최대 40%까지 절감하는 솔루션을 제공한다.",
        "source": "테크 위클리",
        "date": "2025-01-18"
    },
    {
        "title": "온디바이스 AI, 2025년 스마트폰의 새로운 표준으로",
        "content": "클라우드를 거치지 않고 기기 자체에서 AI 연산을 수행하는 '온디바이스 AI'가 2025년 기술 업계의 최대 화두로 떠올랐다. 온디바이스 AI는 빠른 응답 속도, 강화된 개인정보 보호, 인터넷 연결 없는 AI 기능 사용 등의 장점을 가진다.",
        "source": "모바일 월드",
        "date": "2025-01-20"
    }
]

news_documents = [
    Document(
        page_content=article["content"],
        metadata={"title": article["title"], "source": article["source"], "date": article["date"]}
    )
    for article in news_articles
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(news_documents)

news_vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="news_collection"
)

print("벡터 DB 생성 완료!")

---

## 퀴즈 정답

---

### Q1. LLM 한계 이해하기

**문제**: LLM의 4가지 주요 한계(환각, 지식 단절, 개인정보 부재, 출처 불명확) 중 RAG로 해결할 수 있는 것들을 설명하고, 각각 어떻게 해결되는지 서술하세요.

In [ ]:
# 정답
answer_q1 = """
RAG로 해결 가능한 LLM 한계:

1. 환각 (Hallucination) - 해결 가능
   - 해결 방법: RAG는 검색된 실제 문서를 Context로 제공하므로, 
     LLM이 근거 없는 정보를 생성하지 않고 주어진 문서 내용만 기반으로 답변
   - 프롬프트에 "Context에 없으면 모른다고 답변하라"는 규칙 추가로 더욱 효과적

2. 지식 단절 (Knowledge Cutoff) - 해결 가능
   - 해결 방법: 최신 문서를 벡터 DB에 지속적으로 업데이트하면
     모델 재학습 없이도 최신 정보에 접근 가능
   - 예: 어제 발표된 뉴스도 DB에 추가하면 바로 답변 가능

3. 개인/도메인 정보 부재 - 해결 가능
   - 해결 방법: 사내 문서, 내부 규정, 고객 데이터 등을
     벡터 DB에 저장하면 비공개 정보에 대한 답변 가능
   - 예: 회사 휴가 정책, 제품 매뉴얼 등

4. 출처 불명확 - 해결 가능
   - 해결 방법: 검색된 문서의 메타데이터(출처, 날짜 등)를
     함께 반환하여 답변의 근거를 명확히 제시
   - 프롬프트에 "출처를 명시하라"는 지시사항 추가

결론: RAG는 LLM의 4가지 주요 한계를 모두 해결하거나 크게 완화할 수 있는 
효과적인 방법입니다. 특히 기업 환경에서 내부 데이터 기반 AI 서비스 구축에 필수적입니다.
"""

print(answer_q1)

**풀이 설명**

- **접근 방법**: RAG의 핵심 원리(외부 지식 검색 + LLM 생성)를 이해하고, 각 한계가 어떻게 극복되는지 연결
- **핵심 개념**: RAG는 LLM에게 "오픈북"을 제공하여 기억(학습 데이터)에만 의존하지 않게 함
- **실무 팁**: 실제 서비스에서는 검색 품질(Retrieval Quality)이 전체 성능을 좌우하므로 청킹, 임베딩, 검색 전략에 주의

---

### Q2. RAG 파이프라인 이해하기

**문제**: RAG의 3단계 파이프라인(인덱싱, 검색, 생성)을 각각 한 문장으로 설명하고, 각 단계에서 어떤 도구가 사용되는지 서술하세요.

In [ ]:
# 정답
answer_q2 = """
RAG의 3단계 파이프라인:

1. 인덱싱 (Indexing) - 사전 준비 단계
   - 설명: 문서를 작은 청크로 나누고, 임베딩 모델로 벡터화하여 벡터 DB에 저장하는 과정
   - 사용 도구:
     * 텍스트 분할기: RecursiveCharacterTextSplitter (LangChain)
     * 임베딩 모델: OpenAI text-embedding-3-small, sentence-transformers
     * 벡터 DB: Chroma, FAISS, Pinecone

2. 검색 (Retrieval) - 실시간 단계
   - 설명: 사용자 질문을 임베딩하고, 벡터 DB에서 가장 유사한 문서 청크를 찾아 반환하는 과정
   - 사용 도구:
     * 임베딩 모델: 인덱싱과 동일한 모델 사용 (중요!)
     * 벡터 DB 검색: similarity_search, as_retriever (LangChain)
     * 유사도 측정: 코사인 유사도 (가장 일반적)

3. 생성 (Generation) - 실시간 단계
   - 설명: 검색된 문서를 Context로 제공하고, LLM이 질문에 대한 답변을 생성하는 과정
   - 사용 도구:
     * LLM: GPT-4, Claude, LLaMA 등
     * 프롬프트 템플릿: ChatPromptTemplate (LangChain)
     * 체인: create_retrieval_chain, create_stuff_documents_chain (LangChain)

전체 파이프라인 도구 요약:
- LangChain: 전체 파이프라인 오케스트레이션
- OpenAI: 임베딩 + LLM
- Chroma: 벡터 저장 및 검색
"""

print(answer_q2)

**풀이 설명**

- **접근 방법**: 각 단계의 목적과 역할을 명확히 구분하고, 실제 사용되는 도구 매핑
- **핵심 개념**: 인덱싱은 "사전 준비", 검색/생성은 "실시간 처리"
- **주의사항**: 인덱싱과 검색에서 반드시 같은 임베딩 모델을 사용해야 함

---

### Q3. 임베딩 직접 사용하기

**문제**: 아래 세 문장의 임베딩을 생성하고, 코사인 유사도를 계산하세요. 가장 유사한 문장 쌍은 무엇인가요?

In [ ]:
# 정답
sentences = [
    "머신러닝은 데이터로부터 패턴을 학습합니다.",
    "딥러닝은 인공신경망을 활용한 기계학습입니다.",
    "오늘 점심은 김치찌개를 먹었습니다."
]

# 코사인 유사도 함수
def cosine_similarity(vec1, vec2):
    """두 벡터의 코사인 유사도 계산"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 임베딩 생성
embeddings = [embedding_model.embed_query(s) for s in sentences]

# 유사도 계산 및 결과 저장
similarities = []
pairs = []

print("문장 간 코사인 유사도:")
print("=" * 70)

for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        sim = cosine_similarity(embeddings[i], embeddings[j])
        similarities.append(sim)
        pairs.append((i, j))
        print(f"\n문장 {i+1} vs 문장 {j+1}: {sim:.4f}")
        print(f"  '{sentences[i]}'")
        print(f"  '{sentences[j]}'")

# 가장 유사한 쌍 찾기
max_idx = np.argmax(similarities)
most_similar_pair = pairs[max_idx]

print("\n" + "=" * 70)
print(f"\n가장 유사한 문장 쌍: 문장 {most_similar_pair[0]+1}과 문장 {most_similar_pair[1]+1}")
print(f"유사도: {similarities[max_idx]:.4f}")
print(f"  - '{sentences[most_similar_pair[0]]}'")
print(f"  - '{sentences[most_similar_pair[1]]}'")

**풀이 설명**

- **접근 방법**: 각 문장을 임베딩한 후, 모든 쌍에 대해 코사인 유사도 계산
- **핵심 개념**: 의미가 비슷한 문장(머신러닝, 딥러닝)은 벡터 공간에서 가까이 위치
- **예상 결과**: 문장 1(머신러닝)과 문장 2(딥러닝)가 가장 유사 (둘 다 AI/ML 관련)
- **실무 팁**: 유사도가 0.8 이상이면 매우 유사, 0.3 이하면 관련 없음으로 볼 수 있음

---

### Q4. 벡터 DB 검색하기

**문제**: 본문에서 생성한 `news_vectordb`를 사용하여 "클라우드 비용"에 관한 문서를 검색하고, 결과를 출력하세요.

In [ ]:
# 정답
query = "클라우드 비용"

# 유사도 검색 실행
results = news_vectordb.similarity_search(query, k=3)

print(f"검색 쿼리: '{query}'")
print("\n검색 결과:")
print("=" * 70)

for i, doc in enumerate(results, 1):
    print(f"\n[{i}] 제목: {doc.metadata.get('title', 'N/A')}")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print(f"    내용: {doc.page_content[:150]}...")

# 추가: 유사도 점수와 함께 검색
print("\n" + "=" * 70)
print("\n유사도 점수와 함께 검색:")

results_with_scores = news_vectordb.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"\n[{i}] 거리 점수: {score:.4f} (낮을수록 유사)")
    print(f"    제목: {doc.metadata.get('title', 'N/A')}")

**풀이 설명**

- **접근 방법**: `similarity_search()` 메서드에 쿼리와 k값 전달
- **핵심 개념**: 벡터 DB는 쿼리를 임베딩하고 저장된 벡터들과 유사도 비교
- **예상 결과**: "데이터브릭스-옵티마이즈" 관련 문서가 상위에 검색됨 (클라우드 비용 최적화 내용)
- **실무 팁**: k값이 클수록 더 많은 문서가 반환되지만, 관련성 낮은 문서도 포함될 수 있음

---

### Q5. 유사도 검색 이해하기

**문제**: `similarity_search`와 `similarity_search_with_score`의 차이점을 설명하고, 유사도 점수(score)가 의미하는 바를 서술하세요.

In [ ]:
# 정답
answer_q5 = """
similarity_search vs similarity_search_with_score 비교:

1. similarity_search(query, k)
   - 반환값: List[Document]
   - 유사도가 높은 상위 k개 문서만 반환
   - 유사도 점수는 반환하지 않음
   - 사용 시점: 단순히 관련 문서만 필요할 때

2. similarity_search_with_score(query, k)
   - 반환값: List[Tuple[Document, float]]
   - 문서와 함께 유사도 점수(거리)도 반환
   - 점수로 검색 품질을 판단할 수 있음
   - 사용 시점: 검색 결과의 신뢰도를 평가해야 할 때

유사도 점수(Score)의 의미:
- Chroma에서 반환하는 score는 "거리(distance)" 값
- 값이 낮을수록 더 유사함 (0에 가까울수록 좋음)
- 값이 높을수록 덜 유사함

점수 해석 가이드라인 (대략적):
- 0.0 ~ 0.3: 매우 유사 (높은 관련성)
- 0.3 ~ 0.5: 유사 (관련 있음)
- 0.5 ~ 0.7: 보통 (약간 관련)
- 0.7 이상: 관련성 낮음

실무 활용:
- 임계값(threshold) 설정: score > 0.5인 결과는 제외
- 답변 신뢰도 표시: "관련 문서를 찾지 못했습니다" 메시지 출력
- 검색 품질 모니터링: 평균 점수 추적
"""

print(answer_q5)

# 실제 비교 예시
print("\n실제 비교 예시:")
print("=" * 70)

query = "AI 기술"

# similarity_search
results1 = news_vectordb.similarity_search(query, k=2)
print(f"\nsimilarity_search 결과 (반환 타입: {type(results1)})")
for doc in results1:
    print(f"  - {doc.metadata['title'][:30]}...")

# similarity_search_with_score
results2 = news_vectordb.similarity_search_with_score(query, k=2)
print(f"\nsimilarity_search_with_score 결과 (반환 타입: {type(results2)})")
for doc, score in results2:
    print(f"  - {doc.metadata['title'][:30]}... (거리: {score:.4f})")

**풀이 설명**

- **접근 방법**: 두 메서드의 반환 타입과 활용 목적 비교
- **핵심 개념**: score는 거리(distance)이므로 낮을수록 유사
- **주의사항**: 벡터 DB마다 score 계산 방식이 다를 수 있음 (코사인, 유클리드 등)
- **실무 팁**: threshold 기반 필터링으로 관련성 낮은 결과 제외

---

### Q6. 청킹 전략 이해하기

**문제**: `chunk_size`와 `chunk_overlap`의 역할을 설명하고, 각각을 너무 크게/작게 설정했을 때의 문제점을 서술하세요.

In [ ]:
# 정답
answer_q6 = """
chunk_size와 chunk_overlap의 역할:

1. chunk_size (청크 크기)
   - 역할: 각 청크의 최대 문자 수 (또는 토큰 수)
   - 권장값: 500 ~ 1000 글자
   
   너무 크게 설정 (예: 5000):
   - 문제점:
     * 검색 결과에 불필요한 정보 포함 (노이즈 증가)
     * LLM 토큰 한도 초과 위험
     * 비용 증가 (더 많은 토큰 사용)
   
   너무 작게 설정 (예: 50):
   - 문제점:
     * 문맥 손실 (의미 파악 어려움)
     * 중요 정보가 여러 청크에 분산
     * 검색 정확도 저하

2. chunk_overlap (청크 중복)
   - 역할: 연속된 청크 간 공유하는 문자 수
   - 권장값: chunk_size의 10~20% (예: 50~100)
   
   너무 크게 설정 (예: chunk_size의 50%):
   - 문제점:
     * 중복 데이터 과다 저장 (저장 공간 낭비)
     * 검색 시 동일 정보 중복 반환
     * 인덱싱 시간 증가
   
   너무 작게 설정 (예: 0):
   - 문제점:
     * 문장/문단 경계에서 정보 단절
     * 연속된 문맥 파악 어려움
     * 중요 정보가 청크 경계에서 잘림

실무 권장 설정:
- 일반 문서: chunk_size=500, chunk_overlap=50
- 긴 문서: chunk_size=1000, chunk_overlap=100
- 대화/짧은 텍스트: chunk_size=200, chunk_overlap=30
"""

print(answer_q6)

# 실제 비교 예시
print("\n청킹 설정에 따른 결과 비교:")
print("=" * 70)

sample_text = """인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력을 컴퓨터 프로그램으로 실현한 기술입니다.
머신러닝은 AI의 한 분야로, 데이터로부터 패턴을 학습합니다.
딥러닝은 머신러닝의 한 종류로, 인공신경망을 사용합니다.
자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하게 하는 기술입니다."""

# 청크 크기 비교
for chunk_size, overlap in [(50, 0), (150, 30), (300, 50)]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks = splitter.split_text(sample_text)
    print(f"\nchunk_size={chunk_size}, overlap={overlap}: {len(chunks)}개 청크")
    for i, chunk in enumerate(chunks, 1):
        print(f"  [{i}] {chunk[:40]}... ({len(chunk)}글자)")

**풀이 설명**

- **접근 방법**: 각 파라미터의 목적을 이해하고, 극단적인 설정의 부작용 설명
- **핵심 개념**: 청킹은 "정보의 원자 단위"를 결정하는 중요한 과정
- **실무 팁**: 문서 유형에 따라 다른 설정 적용, 테스트를 통해 최적값 찾기

---

### Q7. 프롬프트 템플릿 수정하기

**문제**: 본문의 뉴스 Q&A 봇 프롬프트를 수정하여, 답변 시 반드시 "핵심 요약", "상세 설명", "출처" 세 부분으로 구성되도록 만드세요.

In [ ]:
# 정답
structured_prompt = ChatPromptTemplate.from_template("""
당신은 'AI 뉴스 전문 앵커'입니다. 
최신 AI 뉴스에 대한 질문에 체계적으로 답변해주세요.

규칙:
1. 주어진 Context에 있는 정보만 사용하세요.
2. 답변은 반드시 아래 형식을 따르세요.
3. Context에 답이 없으면 각 섹션에 "정보 없음"이라고 작성하세요.

[Context]
{context}

[질문]
{input}

[답변 형식]

### 핵심 요약
(1-2문장으로 핵심 답변을 작성하세요)

### 상세 설명
(구체적인 내용, 숫자, 데이터를 포함하여 설명하세요)

### 출처
(참조한 뉴스의 제목, 출처, 날짜를 명시하세요)
""")

# 새로운 RAG 체인 생성
news_retriever = news_vectordb.as_retriever(search_kwargs={"k": 3})
structured_document_chain = create_stuff_documents_chain(llm, structured_prompt)
structured_rag_chain = create_retrieval_chain(news_retriever, structured_document_chain)

# 테스트
test_question = "데이터브릭스가 인수한 회사에 대해 알려줘"
response = structured_rag_chain.invoke({"input": test_question})

print(f"질문: {test_question}")
print("\n" + "=" * 70)
print("\n답변:")
print(response['answer'])

**풀이 설명**

- **접근 방법**: 프롬프트에 구체적인 답변 형식(템플릿) 제시
- **핵심 개념**: LLM은 명확한 지시사항을 따르므로, 원하는 출력 형식을 구체적으로 명시
- **대안**: JSON 형식, 마크다운 표, 번호 목록 등 다양한 형식 가능
- **실무 팁**: Few-shot 예시를 추가하면 더욱 일관된 출력 가능

---

### Q8. RAG 체인 구현하기

**문제**: 아래 문서들을 사용하여 새로운 RAG 체인을 구현하고, 질문에 답변하세요.

In [ ]:
# 정답

# 1. 문서 준비
doc_contents = [
    "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다.",
    "파이썬은 가독성이 좋고 배우기 쉬워 초보자에게 추천됩니다.",
    "데이터 과학과 AI 분야에서 파이썬이 가장 많이 사용됩니다.",
    "자바스크립트는 웹 개발의 표준 언어입니다."
]

python_docs = [
    Document(page_content=content, metadata={"topic": "programming"})
    for content in doc_contents
]

# 2. 벡터 DB 생성
python_vectordb = Chroma.from_documents(
    documents=python_docs,
    embedding=embedding_model,
    collection_name="python_docs"
)

# 3. 프롬프트 템플릿
python_prompt = ChatPromptTemplate.from_template("""
당신은 프로그래밍 언어 전문가입니다.
주어진 Context를 바탕으로 질문에 답변하세요.

규칙:
1. Context에 있는 정보만 사용하세요.
2. 간결하고 명확하게 답변하세요.
3. Context에 없는 정보는 언급하지 마세요.

[Context]
{context}

[질문]
{input}

[답변]
""")

# 4. RAG 체인 생성
python_retriever = python_vectordb.as_retriever(search_kwargs={"k": 3})
python_doc_chain = create_stuff_documents_chain(llm, python_prompt)
python_rag_chain = create_retrieval_chain(python_retriever, python_doc_chain)

# 5. 질문에 답변
question = "파이썬은 누가 만들었고, 어떤 분야에서 많이 사용돼?"
response = python_rag_chain.invoke({"input": question})

print(f"질문: {question}")
print(f"\n답변: {response['answer']}")

print("\n" + "=" * 70)
print("\n사용된 Context:")
for i, doc in enumerate(response['context'], 1):
    print(f"  [{i}] {doc.page_content}")

**풀이 설명**

- **접근 방법**: RAG 구현 5단계 (문서 준비 -> 벡터 DB -> 프롬프트 -> 체인 -> 실행)
- **핵심 개념**: 각 단계별로 필요한 컴포넌트를 올바르게 연결
- **주의사항**: Document 객체 생성 시 metadata 추가 권장
- **실무 팁**: 실제 서비스에서는 벡터 DB를 영구 저장(persist)해야 함

---

### Q9. RAG 시스템 분석하기

**문제**: RAG 시스템에서 검색 결과가 좋지 않을 때 (관련 없는 문서가 검색될 때) 개선할 수 있는 방법 3가지를 서술하세요.

In [ ]:
# 정답
answer_q9 = """
RAG 검색 품질 개선 방법 3가지:

1. 청킹 전략 개선
   - 문제: 청크가 너무 크거나 작아 관련 정보가 제대로 검색되지 않음
   - 해결책:
     * chunk_size 조정: 문서 특성에 맞게 최적값 찾기 (테스트 필요)
     * chunk_overlap 증가: 문맥 연결성 향상
     * 의미 기반 청킹: 문단/섹션 경계를 존중하는 분할
     * 메타데이터 추가: 제목, 날짜, 카테고리 정보 포함

2. 임베딩 모델 변경
   - 문제: 현재 임베딩 모델이 도메인/언어에 최적화되지 않음
   - 해결책:
     * 도메인 특화 모델 사용: 의료, 법률 등 전문 분야용 모델
     * 다국어 모델: 한국어에 강한 multilingual-e5 등
     * 더 큰 차원의 모델: text-embedding-3-small -> large
     * Fine-tuned 모델: 자체 데이터로 임베딩 모델 미세조정

3. 검색 전략 개선
   - 문제: 단순 유사도 검색만으로는 한계가 있음
   - 해결책:
     * Hybrid Search: 키워드 검색 + 벡터 검색 결합
     * Re-ranking: 1차 검색 후 LLM으로 재순위화
     * Query Expansion: 질문을 여러 버전으로 확장하여 검색
     * MMR (Maximal Marginal Relevance): 다양성 고려한 결과 선택
     * Threshold 적용: 유사도 임계값 이하 결과 제외

추가 개선 방법:
- 데이터 품질 개선: 오타, 중복, 노이즈 제거
- 질문 전처리: 질문 명확화, 키워드 추출
- 평가 시스템 구축: 검색 성능 정량적 측정
"""

print(answer_q9)

**풀이 설명**

- **접근 방법**: RAG 파이프라인의 각 단계(청킹, 임베딩, 검색)에서 개선점 도출
- **핵심 개념**: 검색 품질이 RAG 전체 성능의 핵심 (Garbage In, Garbage Out)
- **실무 팁**: 문제 진단 먼저! 어디서 성능 저하가 발생하는지 파악 후 개선

---

### Q10. 나만의 Q&A 봇 만들기

**문제**: 아래 주제 중 하나를 선택하여 최소 5개의 문서를 작성하고, Q&A 봇을 구현하세요.

In [ ]:
# 정답 예시: 회사 FAQ 봇

# 1. 문서 준비 (회사 FAQ)
company_docs = [
    Document(
        page_content="연차 휴가는 입사 1년 후부터 15일이 부여됩니다. 3년차부터 매년 1일씩 추가되며 최대 25일까지 사용할 수 있습니다.",
        metadata={"category": "휴가", "topic": "연차"}
    ),
    Document(
        page_content="병가는 유급으로 연간 10일까지 사용할 수 있습니다. 3일 이상 연속 사용 시 진단서를 제출해야 합니다.",
        metadata={"category": "휴가", "topic": "병가"}
    ),
    Document(
        page_content="재택근무는 주 2회까지 가능합니다. 화상회의 참석 시 카메라 ON이 필수이며, 응답 지연은 30분 이내여야 합니다.",
        metadata={"category": "근무", "topic": "재택근무"}
    ),
    Document(
        page_content="점심시간은 12시부터 1시까지이며, 사내 식당 이용 시 월 10만원까지 회사가 지원합니다.",
        metadata={"category": "복지", "topic": "식대"}
    ),
    Document(
        page_content="교육비는 연간 300만원까지 지원됩니다. 업무 관련 온라인 강의, 자격증 시험비, 컨퍼런스 참가비 등에 사용할 수 있습니다.",
        metadata={"category": "복지", "topic": "교육비"}
    ),
    Document(
        page_content="경조사비는 본인 결혼 100만원, 직계가족 상 50만원, 배우자 직계가족 상 30만원이 지급됩니다.",
        metadata={"category": "복지", "topic": "경조사"}
    ),
]

# 2. 벡터 DB 생성
company_vectordb = Chroma.from_documents(
    documents=company_docs,
    embedding=embedding_model,
    collection_name="company_faq"
)

print(f"회사 FAQ 벡터 DB 생성 완료! ({len(company_docs)}개 문서)")

In [ ]:
# 3. 프롬프트 템플릿
company_prompt = ChatPromptTemplate.from_template("""
당신은 '단테 컴퍼니'의 친절한 HR 챗봇입니다.
직원들의 회사 정책에 대한 질문에 명확하고 친근하게 답변해주세요.

규칙:
1. 주어진 Context의 정보만 사용하세요.
2. 구체적인 숫자(일수, 금액)를 포함하여 답변하세요.
3. Context에 없는 내용은 "해당 정보는 HR팀에 문의해주세요."라고 안내하세요.
4. 친근한 말투를 사용하세요.

[Context]
{context}

[질문]
{input}

[답변]
""")

# 4. RAG 체인 생성
company_retriever = company_vectordb.as_retriever(search_kwargs={"k": 3})
company_doc_chain = create_stuff_documents_chain(llm, company_prompt)
company_rag_chain = create_retrieval_chain(company_retriever, company_doc_chain)

print("회사 FAQ 봇 준비 완료!")

In [ ]:
# 5. Q&A 테스트
test_questions = [
    "연차는 며칠이나 받을 수 있어요?",
    "재택근무 할 때 규칙이 뭐에요?",
    "교육비 지원은 얼마나 되나요?",
    "야근 수당은 어떻게 되나요?"  # Context에 없는 질문
]

print("회사 FAQ 봇 테스트:")
print("=" * 70)

for q in test_questions:
    response = company_rag_chain.invoke({"input": q})
    print(f"\n질문: {q}")
    print(f"답변: {response['answer']}")
    print("-" * 70)

In [ ]:
# 대화형 함수
def ask_hr_bot(question):
    """회사 FAQ 봇에게 질문하기"""
    response = company_rag_chain.invoke({"input": question})
    
    print(f"\n[직원 질문] {question}")
    print(f"\n[HR 봇 답변] {response['answer']}")
    
    print(f"\n[참조 정보]")
    for doc in response['context']:
        print(f"  - [{doc.metadata.get('category', 'N/A')}] {doc.page_content[:50]}...")
    
    return response

# 추가 테스트
ask_hr_bot("결혼하면 축의금을 얼마나 받을 수 있어요?")

**풀이 설명**

- **접근 방법**: 실제 회사 FAQ 시나리오를 구현하여 RAG 전체 파이프라인 경험
- **핵심 개념**: 도메인 특화 프롬프트 설계, 메타데이터 활용
- **확장 방향**: 
  - 더 많은 문서 추가
  - 카테고리별 검색 필터링
  - 대화 히스토리(Memory) 추가
  - 웹 인터페이스 연동
- **실무 팁**: 실제 서비스에서는 사용자 피드백을 수집하여 지속적으로 개선

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| LLM 한계 | 환각, 지식 단절, 내부정보 부재 | RAG로 극복 |
| RAG | 검색 + 생성 조합 | 사내 문서 기반 AI |
| 임베딩 | 텍스트 -> 벡터 변환 | 의미 기반 검색 |
| 벡터 DB | 벡터 저장 및 검색 | Chroma, FAISS |
| 유사도 검색 | 코사인 유사도 기반 | Top-K 문서 반환 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 권장 설정 |
|------|----------|----------|
| 청킹 | 긴 문서 분할 | chunk_size: 500-1000 |
| 오버랩 | 문맥 연결 유지 | chunk_overlap: 50-100 |
| LangChain | RAG 파이프라인 구축 | 체인 패턴 활용 |
| 프롬프트 | Context + 지시사항 | 명확한 규칙 제시 |

### 실무 체크리스트

```
RAG 구현 시 확인사항:
[ ] 인덱싱과 검색에 동일한 임베딩 모델 사용
[ ] 적절한 chunk_size와 overlap 설정
[ ] 프롬프트에 "Context에 없으면 모른다고 답변" 규칙 포함
[ ] 메타데이터로 출처 정보 관리
[ ] 검색 결과 품질 테스트
```